<a href="https://colab.research.google.com/github/mohamedatef773/Data-Science/blob/main/text_generation_arbic_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np 
import os 
import time
import spacy
import re

# NGrams

# Search for data
#### Arabic-News
##### Arabic News for language modelling collected from

##### BBC Arabic 
##### EuroNews
##### Aljazeera
##### CNN Arabic
##### RT Arabic

In [ ]:
# read the data  
text=open('/content/drive/MyDrive/aljazeera.net_20190419_titles.txt','rb').read().decode(encoding='utf-8')


In [ ]:
import re
def preprocess(text):  
    text = re.sub(r'[^\w\s]', '', text)
    return "".join(text)


In [ ]:
text=preprocess(text)

In [ ]:
text[:250]

In [ ]:
# the unique characters in the file 
vocab=sorted(set(text))
print('{} unique characters'.format(len(vocab)))
vocab

In [ ]:
# The i:j part of the comprehension creates a new key-value pair in the dictionary, with the key being the element j and the value being the index i.
char_idx={j:i for i,j in enumerate(vocab)}
idx_char=np.array(vocab)
# convert the text to int
text_as_int=np.array([char_idx[c] for c in text])


In [ ]:
char_idx

In [ ]:
text_as_int[:255]

In [ ]:
# show ho the first 20 chararacters from the text are mapped to integers
print('{}--------- characters mapped to int--------> {}'.format(repr(text[:13]),text_as_int[:13]))

In [ ]:
# the maximum lenght sentence for the single input in characters  
seq_length=100
# create training exaples / targets
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)


In [ ]:
# to make the input the same size in the character  and add the one character to make the  traget 
squences=char_dataset.batch(seq_length+1,drop_remainder=True)


function returns a printable representation of the given object.


In [ ]:
for item in squences.take(5):
  print(repr(''.join(idx_char[item.numpy()])),item,len(item))

In [ ]:
def split_input_target(chunk):
  input_text=chunk[:-1]
  traget_text=chunk[1:]
  return input_text,traget_text

In [ ]:
dataset=squences.map(split_input_target)

In [ ]:
for i ,j in dataset.take(1):
  print('input data','',''.join(idx_char[i.numpy()]))
  print('Taraget data','',''.join(idx_char[j.numpy()]))

In [ ]:
len(dataset)

In [ ]:
batch_size=64
#  BUFFER_SIZE (which starts empty but has enough room to store that many elements). 
BUFFER_SIZE=10000
# make the  dataset shuffle  and put in the batch
dataset=dataset.shuffle(BUFFER_SIZE).batch(batch_size,drop_remainder=True)

In [ ]:
vocb_size=len(vocab)
embedding_dim=256
ruu_units=1024

In [ ]:
def build_model(vocb_size,embedding_dim,ruu_units,batch_size):
  model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocb_size,embedding_dim,batch_input_shape=[batch_size,None]),
      tf.keras.layers.GRU(ruu_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
      #                                                                                              Xavier/ wights initializations
      tf.keras.layers.Dense(vocb_size)
              ])
  return model

In [ ]:
model=build_model(vocb_size,embedding_dim,ruu_units,batch_size)

In [ ]:
model.summary()

In [ ]:
# Use this crossentropy loss function when there are two or more label classes.

In [ ]:
model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy')


In [ ]:
history=model.fit(dataset,epochs=20)

In [ ]:
text_as_int[:255]